# Make a catalog of SEDs

In [40]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
import copy
output_notebook()

Loading BokehJS ...

First we initialize an empty catalog.

In [44]:
# Make an SED catalog
cat = sed.SEDCatalog()

Now let's make a few SEDs

In [45]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [46]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    s2.make_sed()
    cat.add_SED(s2)

Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!


Then the aggregated results look like this:

In [47]:
cat.results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11a68d4e0>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,542.78,29.46,1.8423835404296787,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.2940033546053776e+32,3.58942636659248e+31,-1.065,0.047,7.419,0.06,4.5,0.07,0.8896913720017506,0.0,3335.0,157.0,<SEDkit.sed.SED object at 0x11a2f4d30>,20.486,0.073,60353.831,6551.546,17.779,0.006,52378.735,5685.83,16.431,0.004,48407.39,5254.732,15.843,0.004,46675.083,5066.686,15.514,0.005,45705.815,4961.47,14.391,0.029,42397.344,4602.328,13.677,0.032,40293.827,4373.987,13.535,0.034,39875.481,4328.574,13.55,0.027,39919.672,4333.371,13.581,0.035,40011.002,4343.285,12.884,0.0,37957.569,4120.38
2MASS 2618328861,6000.0,4000.0,604.18,36.5,1.655142783544274,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.081398997241159e+32,4.946615589771715e+31,-0.972,0.053,7.186,0.066,4.5,0.07,0.8896913720017506,0.0,3518.0,172.0,<SEDkit.sed.SED object at 0x11a4d36d8>,20.486,0.073,74780.757,9035.38,17.779,0.006,64899.301,7841.453,16.431,0.004,59978.65,7246.916,15.843,0.004,57832.253,6987.577,15.514,0.005,56631.293,6842.471,14.391,0.029,52531.967,6347.171,13.677,0.032,49925.628,6032.26,13.535,0.034,49407.28,5969.631,13.55,0.027,49462.036,5976.247,13.581,0.035,49575.196,5989.919,12.884,0.0,47030.913,5682.506
2MASS 2618328862,6000.0,4000.0,564.23,31.84,1.7723320109810539,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.559497656325819e+32,4.031564698151861e+31,-1.032,0.049,7.335,0.062,4.5,0.07,0.8896913720017506,0.0,3400.0,162.0,<SEDkit.sed.SED object at 0x119f36a20>,20.486,0.073,65218.306,7360.655,17.779,0.006,56600.423,6388.024,16.431,0.004,52308.991,5903.686,15.843,0.004,50437.061,5692.416,15.514,0.005,49389.671,5574.206,14.391,0.029,45814.539,5170.711,13.677,0.032,43541.481,4914.

We can also plot the results by passing the `plot` method two column names.

In [50]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

Or plot a CMD!

In [ ]:
show(cat.plot('distance', 'Teff'))

We can then retrieve the SED for a particular source like so:

In [7]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()

And save and load from file.

In [11]:
cat.save('/Users/jfilippazzo/Desktop/test.p')

PicklingError: Can't pickle <class 'bokeh.core.enums.Enumeration'>: it's not the same object as bokeh.core.enums.Enumeration

In [10]:
new = sed.SEDCatalog().load('/Users/jfilippazzo/Desktop/test.p')

TypeError: a bytes-like object is required, not 'str'

In [29]:
cat.results[:0]

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
